<a href="https://colab.research.google.com/github/Fatemejamalii/Mizani/blob/main/ID_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In the name of God

inpainting(id and atrr)

###nvidia-smi

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
!nvidia-smi

Sat Sep 18 10:59:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |    345MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###preparing CelebA dataset

In [2]:
%cd /content

import os
os.environ['KAGGLE_USERNAME'] = "fatimajamali" # username from the json file
os.environ['KAGGLE_KEY'] = "8bdd2d734bb7389bd19e10afd81bc741" # key from the json file
!kaggle datasets download -d lamsimon/celebahq 
#!unzip /content/celebahq.zip
from zipfile import ZipFile
with ZipFile('/content/celebahq.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content')

/content
100% 2.54G/2.55G [00:16<00:00, 188MB/s]
100% 2.55G/2.55G [00:16<00:00, 162MB/s]


In [4]:
!unzip  /content/drive/MyDrive/mizani/CelebA_dataset/train_mask.zip -d /content/celeba_hq/train

Streaming output truncated to the last 5000 lines.
  inflating: /content/celeba_hq/train/train_mask/066773.jpg  
  inflating: /content/celeba_hq/train/train_mask/066835.jpg  
  inflating: /content/celeba_hq/train/train_mask/066830.jpg  
  inflating: /content/celeba_hq/train/train_mask/066790.jpg  
  inflating: /content/celeba_hq/train/train_mask/066966.jpg  
  inflating: /content/celeba_hq/train/train_mask/066810.jpg  
  inflating: /content/celeba_hq/train/train_mask/066922.jpg  
  inflating: /content/celeba_hq/train/train_mask/066826.jpg  
  inflating: /content/celeba_hq/train/train_mask/066990.jpg  
  inflating: /content/celeba_hq/train/train_mask/066958.jpg  
  inflating: /content/celeba_hq/train/train_mask/066807.jpg  
  inflating: /content/celeba_hq/train/train_mask/066795.jpg  
  inflating: /content/celeba_hq/train/train_mask/066899.jpg  
  inflating: /content/celeba_hq/train/train_mask/066956.jpg  
  inflating: /content/celeba_hq/train/train_mask/066767.jpg  
  inflating: /conte

### git clone Mizani project

In [5]:
!git clone https://github.com/Fatemejamalii/Mizani.git

Cloning into 'Mizani'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 374 (delta 188), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (374/374), 167.87 KiB | 6.71 MiB/s, done.
Resolving deltas: 100% (188/188), done.


#### install packeges

In [6]:
#!pip uninstall tensorflow==2.5.0 --yes
#!pip install tensorflow==3.1.0
!pip install h5py==2.10.0
import tensorflow as tf
print(tf.__version__)
!pip install tensorflow_addons
!pip install mtcnn
!pip install face-alignment
!pip install wandb

     |████████████████████████████████| 2.9 MB 8.5 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.


2.6.0
     |████████████████████████████████| 1.1 MB 8.2 MB/s 
     |████████████████████████████████| 2.3 MB 7.6 MB/s 
  Created wheel for face-alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28239 sha256=8e1df23327e339e3a0d621c4d70e4708d984f267648f484214a17e0d79e31d23
  Stored in directory: /root/.cache/pip/wheels/c9/ba/4d/2d368f55e5f929f9472da59e356fbdf1483f885de80a5bc620
Successfully built face-alignment
     |████████████████████████████████| 1.7 MB 7.2 MB/s 
     |████████████████████████████████| 97 kB 8.7 MB/s 
     |████████████████████████████████| 133 kB 85.9 MB/s 
     |████████████████████████████████| 180 kB 96.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=2b272a99b42987d9a5e28f2149f73f633f45bdec76f59769acd8a44f5b966f3d
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel 

### Train


In [9]:
%cd /..
%cd /content/Mizani
!python main.py -h

/
/content/Mizani
usage: main.py [-h] [--gpu GPU] [--face_detection] [--resolution {256,1024}]
               [--load_checkpoint LOAD_CHECKPOINT] --pretrained_models_path
               PRETRAINED_MODELS_PATH [--const_noise | --no_const_noise]
               [--batch_size BATCH_SIZE] [--reals]
               [--test_real_attr | --no_test_real_attr]
               [--results_dir RESULTS_DIR] [--log_debug] [--debug]
               [--dataset_path DATASET_PATH] [--celeba_path CELEBA_PATH]
               [--wich_dataset WICH_DATASET] [--num_epochs NUM_EPOCHS]
               [--cross_frequency CROSS_FREQUENCY] [--unified]
               [--train_real_attr | --no_train_real_attr]
               [--train_data_size TRAIN_DATA_SIZE] [--id_loss | --no_id_loss]
               [--landmarks_loss | --no_landmarks_loss]
               [--pixel_loss | --no_pixel_loss] [--W_D_loss | --no_W_D_loss]
               [--gp | --no_gp] [--pixel_mask_type {uniform,gaussian}]
               [--pixel_loss_type {

In [ ]:
%cd /..
%cd /content/Mizani
!python main.py celebaa_test --resolution 256 --pretrained_models_path /content/drive/MyDrive/mizani/pretrained --batch_size 2 --cross_frequency 0 --train_data_size 32000 --results_dir /content/drive/MyDrive/mizani/result --dataset_path /content/drive/MyDrive/mizani/Dataset_256 --no_train_real_attr --no_test_real_attr --no_test_with_arcface --celeba_path /content/celeba_hq/train --wich_dataset celeba


Streaming output truncated to the last 5000 lines.
2021-09-18 11:48:08 - Trainer - INFO - G gan loss is 0.000
2021-09-18 11:48:09 - Trainer - INFO - global norm G not gan grad: 0.3482509255409241
2021-09-18 11:48:09 - Trainer - INFO - ---------------------------------------
2021-09-18 11:48:09 - Trainer - INFO - Start training epoch: 106
2021-09-18 11:48:09 - Trainer - INFO - This epoch is same-face
2021-09-18 11:48:09 - Trainer - INFO - train in epoch: 106
2021-09-18 11:48:12 - Trainer - INFO - attr embedding stats- mean: 0.09339, variance: 0.03668
2021-09-18 11:48:12 - Trainer - INFO - w stats- mean: 0.26344, variance: 0.10576
2021-09-18 11:48:12 - Trainer - INFO - g W loss is 0.299
2021-09-18 11:48:12 - Trainer - INFO - fake W logit: [1.0700663 1.037697 ]
2021-09-18 11:48:12 - Trainer - INFO - w_d_gp : 8.11620807647705
2021-09-18 11:48:12 - Trainer - INFO - W_D loss is 2.079
2021-09-18 11:48:12 - Trainer - INFO - real W logit: [-0.08244618 -0.04861762]
2021-09-18 11:48:12 - Trainer 

### Test

In [14]:
%cd /..
%cd /content/Mizani
!python test.py test --pretrained_models_path /content/drive/MyDrive/mizani/pretrained --load_checkpoint /content/drive/MyDrive/mizani/result/training_id_is_trainable_start_from_epoch_45000/weights_for_test --id_dir /content/drive/MyDrive/mizani/Dataset_256/ID --attr_dir /content/drive/MyDrive/mizani/Dataset_256/ID --mask_dir /content/drive/MyDrive/mizani/Dataset_256/mask --output_dir /content/drive/MyDrive/mizani/result/training_id_is_trainable_start_from_epoch_45000/output_test_1 --test_func opt_infer_pairs --epochnum 52000

Streaming output truncated to the last 5000 lines.
loss value is: 46.031044006347656
loss value is: 47.43860626220703
loss value is: 50.38529968261719
loss value is: 47.46215057373047
loss value is: 47.430213928222656
loss value is: 49.03424835205078
loss value is: 45.64080810546875
loss value is: 44.65471649169922
loss value is: 47.05784606933594
loss value is: 47.948974609375
loss value is: 46.879638671875
loss value is: 47.56087875366211
loss value is: 47.67686462402344
loss value is: 44.79908752441406
loss value is: 43.791954040527344
loss value is: 46.065345764160156
loss value is: 49.12041473388672
loss value is: 47.78484344482422
loss value is: 45.738990783691406
loss value is: 44.50634765625
loss value is: 44.41954803466797
loss value is: 44.3145866394043
loss value is: 44.579742431640625
loss value is: 44.56903076171875
loss value is: 45.84627151489258
loss value is: 46.72810363769531
loss value is: 46.140899658203125
loss value is: 44.10491943359375
loss value is: 41.54061889

### Test for celeba dataset

In [ ]:
import os
from PIL import Image
for imagename in os.listdir('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_mask'):
  if imagename[-4:] == '.jpg':
    im = Image.open('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_ID' + '/' + imagename)
    if im.size != (256 , 256) :
      im = im.resize((256,256))
      im.save('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_ID' + '/' + imagename[-10:])


for imagename in os.listdir('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_mask'):
  if imagename[-4:] == '.jpg':
    im = Image.open('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_mask' + '/' + imagename)
    if im.size != (256 , 256) :
      im = im.resize((256,256))
      im.save('/content/drive/MyDrive/mizani/Dataset_256/val_celeba_mask' + '/' + imagename[-10:])



In [ ]:
%cd /..
%cd /content/Mizani
!python test.py test --pretrained_models_path /content/drive/MyDrive/mizani/pretrained --load_checkpoint /content/drive/MyDrive/mizani/result/training_id_is_trainable_start_from_epoch_45000/weights_for_test --id_dir /content/drive/MyDrive/mizani/Dataset_256/val_celeba_ID --attr_dir /content/drive/MyDrive/mizani/Dataset_256/val_celeba_ID --mask_dir /content/drive/MyDrive/mizani/Dataset_256/val_celeba_mask --output_dir /content/drive/MyDrive/mizani/result/training_id_is_trainable_start_from_epoch_45000/output_opt_final_celeba --test_func opt_infer_pairs --epochnum 52000

/
/content/Mizani
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
  0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100% 2/2 [25:57<00:00, 778.95s/it]
